In [ ]:
import pickle
import logging
import pandas as pd
import numpy as np
from numpy import random
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
import emoji
nltk.download('stopwords')
# from bs4 import BeautifulSoup
%matplotlib inline

df = pd.read_csv(r"C:\Users\Aagaaz Kapoor\Desktop\Meetings_KRA\KRA-Preidictive-model-using-Python\Category Sample Data - Sheet4.csv")
df = df[pd.notnull(df['Category'])]
print(df.head(10))

        Category                             Description
0       Training                                    Self
1       Training                               Tat check
2       Training                            Checker tesr
3             IT                                 App dev
4  Miscellaneous         cheque collection from arvind24
5  Miscellaneous  Excel doc for module attribute lusting
6  Miscellaneous                                 Welcome
7  Miscellaneous                                 Welcome
8       Training  Conduct training to the new candidates
9       Training     Conduct training for new candidates


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91921\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def print_plot(index):
    example = df[df.index == index][['Description', 'Category']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Tag:', example[1])
print_plot(10)

Noti check
Tag: Training


In [7]:
print_plot(30)

Suggestion/Bug Report
Tag: IT


In [8]:
import re
from nltk.corpus import stopwords
import emoji

# Define the regex patterns
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')

# Get the default English stopwords
STOPWORDS = set(stopwords.words('english'))

# Add new stop words
# custom_stopwords = {'bug', 'bugs'}
# STOPWORDS.update(custom_stopwords)

# Get all emojis and add them to the stopwords
all_emojis = set(emoji.EMOJI_DATA.keys())
STOPWORDS.update(all_emojis)

def clean_text(text):
    """
    text: a string
    return: modified initial string
    """
    # text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower()  # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text)  # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text)  # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)  # delete stopwords from text
    return text

# Example usage
example_text = "This is a BUG and several bugs, including a bug fix. 😊"
cleaned_text = clean_text(example_text)
print(cleaned_text)


bug several bugs including bug fix


<>:6: SyntaxWarning: invalid escape sequence '\['
<>:6: SyntaxWarning: invalid escape sequence '\['
C:\Users\91921\AppData\Local\Temp\ipykernel_29204\1960360633.py:6: SyntaxWarning: invalid escape sequence '\['
  REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')


In [9]:
df['Category'].apply(lambda x: len(x.split(' '))).sum()

np.int64(60450)

In [10]:
df['Description'] = df['Description'].fillna('')

print(df)

            Category                                       Description
0           Training                                              Self
1           Training                                         Tat check
2           Training                                      Checker tesr
3                 IT                                           App dev
4      Miscellaneous                   cheque collection from arvind24
...              ...                                               ...
45253             IT  33381 | ColdForge | N.A(N.A) | Bug rectification
45254             IT                         Arvind and swastik backup
45255             IT                          New Organisation Created
45256             IT                           Bill book configuration
45257             IT                              ACL-ComplaintID-6097

[45258 rows x 2 columns]


In [11]:
id_to_category = {
    0: 'Training',
    1: 'Planning',
    2: 'Marketing & Sales',
    3: 'Finance',
    4: 'HR',
    5: 'IT',
    6: 'Operations',
    7: 'Logistics',
    8: 'Miscellaneous',
    9: 'Other'
}

In [12]:
category_to_id = {v: k for k, v in id_to_category.items()}

In [13]:
X = df.Description
y = df.Category
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_20newsgroups



# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the pipeline with a Random Forest classifier
rf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier()),
])

# Fit the model
rf.fit(X_train, y_train)

# Predict the test data
y_pred = rf.predict(X_test)

# Calculate and print the accuracy
print('accuracy %s' % accuracy_score(y_pred, y_test))

accuracy 0.8325231992929739


In [15]:
with open('Meetings11.pkl', 'wb') as pickle_file:
    pickle.dump(rf, pickle_file)

In [ ]:
# CODE FOR CALLING THE PICKLE FILE FOR FUTURE USE.
# import pickle

# # Load the model
# with open('sentiment_model.pkl', 'rb') as pickle_file:
#     loaded_model = pickle.load(pickle_file)

# # Use the loaded model for prediction on new data
# new_text = "This movie was fantastic!"
# new_features = vectorizer.transform([new_text])  # Transform new text
# prediction = loaded_model.predict(new_features)
# print("Predicted sentiment:", prediction)


In [16]:
id_to_category = {
    'Training': 'Training',
     'Planning': 'Planning',
    'Marketing & Sales': 'Marketing & Sales',
    'Finance': 'Finance',
    'HR': 'HR',
    'IT': 'IT',
    'Operations': 'Operations',
    'Logistics': 'Logistics',
    'Miscellaneous': 'Miscellaneous',
    'Other': 'Other'
    
}


In [18]:
# Predict top category
def predict_top_category(description, model, vectorizer):
    # Preprocess the description if needed
    # Vectorize the description using the same vectorizer
    X_new = vectorizer.transform([description])
    
    # Predict probabilities for each category
    probas = model.predict_proba(X_new)[0]
    
    # Get index of the top category based on probability
    top_index = probas.argmax()
    
    # Get the category label from the model
    top_category = model.classes_[top_index]
    
    # Get the category name corresponding to the top index
    top_category_name = id_to_category[top_category]
    
    return top_category_name

# Example usage
new_description = "#Bug purchase"
predicted_category = predict_top_category(new_description, rf.named_steps['clf'], rf.named_steps['vect'])
print(f"Predicted category for the description '{new_description}': {predicted_category}")


Predicted category for the description '#Bug purchase': IT


In [88]:
# Predict top 3 categories
def predict_top_3_categories(description, model, vectorizer):
    # Preprocess the description if needed
    # Vectorize the description using the same vectorizer
    X_new = vectorizer.transform([description])
    
    # Predict probabilities for each category
    probas = model.predict_proba(X_new)[0]
    
    # Get indices of top 3 categories based on probabilities
    top_3_indices = probas.argsort()[-3:][::-1]
    
    # Get the category labels from the model
    top_3_categories = model.classes_[top_3_indices]
    
    # Get category names corresponding to the top 3 indices
    top_3_category_names = [id_to_category[idx] for idx in top_3_categories]
    
    return top_3_category_names

# Example usage
new_description = "Development of training programs"
predicted_categories = predict_top_3_categories(new_description, rf.named_steps['clf'], rf.named_steps['vect'])
print(f"Predicted top 3 categories for the description '{new_description}': {predicted_categories}")


Predicted top 3 categories for the description 'Development of training programs': ['IT', 'Marketing & Sales', 'Training']


In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

# Assuming X and y are already defined as features and labels

# Primary model to predict top 3 categories
def predict_top_3_categories(description, model, vectorizer):
    # Preprocess the description if needed
    # Vectorize the description using the same vectorizer
    X_new = vectorizer.transform([description])
    
    # Predict probabilities for each category
    probas = model.predict_proba(X_new)[0]
    
    # Get indices of top 3 categories based on probabilities
    top_3_indices = probas.argsort()[-3:][::-1]
    
    # Get the category labels from the model
    top_3_categories = model.classes_[top_3_indices]
    
    # Get category names corresponding to the top 3 indices
    top_3_category_names = [id_to_category[idx] for idx in top_3_categories]
    
    return top_3_category_names

# Example usage
new_description = "SEO and Blogs - 26 Jun 24"
top_3_categories = predict_top_3_categories(new_description, rf.named_steps['clf'], rf.named_steps['vect'])

print(f"Predicted top 3 categories for the description '{new_description}': {top_3_categories}")

# Filter dataset for top 3 categories
filtered_df = df[df.Category.isin(top_3_categories)]
X_top3 = filtered_df.Description
y_top3 = filtered_df.Category.apply(lambda x: top_3_categories.index(x))

# Secondary model pipeline
secondary_model = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', RandomForestClassifier())
])

# Train secondary model
secondary_model.fit(X_top3, y_top3)

# Predict single category from top 3
X_pred = [new_description] * len(top_3_categories)
y_pred_proba = secondary_model.predict_proba(X_pred)
predicted_single_category_index = y_pred_proba.mean(axis=0).argmax()
predicted_single_category = top_3_categories[predicted_single_category_index]

print(f"Predicted single category from top 3: {predicted_single_category}")

Predicted top 3 categories for the description 'SEO and Blogs - 26 Jun 24': ['Training', 'Marketing & Sales', 'IT']
Predicted single category from top 3: Marketing & Sales


In [20]:
import pandas as pd
import json

# Specify the path to your CSV file
data_file = "C:\\Users\\91921\\OneDrive\\Desktop\\Refined data.csv"
# Read the data from the CSV file
try:
  df = pd.read_csv(data_file)
except FileNotFoundError:
  print("Error: File not found. Please check the path and filename.")
  exit()

# Convert the Pandas DataFrame to a list of dictionaries (suitable for JSON)
json_data = df.to_dict(orient='records')  # 'records' creates a list of dictionaries

# Save the data to a JSON file
with open('data.json', 'w') as json_file:
  json.dump(json_data, json_file, indent=4)  # Optional indentation for readability

print("JSON file created successfully!")


JSON file created successfully!


In [29]:
from flask import Flask, request, jsonify
import pickle

# Load the pickled model
model = pickle.load(open("Meetings11.pkl", "rb"))


# Create a Flask app
app = Flask(__name__)

@app.route("/predict", methods=["POST"])
def predict():
    # Get the JSON data from the request
    data = request.get_json()

    # Assuming 'description' is the key in the JSON data for your prediction
    description = data.get("description")

    # Make prediction using the loaded model
    prediction = model.predict([description])  # Assuming your model takes a list
    predicted_category = prediction[0]  # Extract the first prediction

    # Return the prediction as JSON
    return jsonify({"prediction": predicted_category})

if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1

c:\Users\91921\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [34]:
# from sklearn.model_selection import train_test_split
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.metrics import accuracy_score

# # Vectorize the text using CountVectorizer with max_features=8
# vectorizer = CountVectorizer(stop_words='english', max_features=8)
# X = vectorizer.fit_transform(df['Description'])
# y = df['Category']

# # Train-test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Train the model
# model = MultinomialNB()
# model.fit(X_train, y_train)

# # Predict on the test set
# y_pred = model.predict(X_test)

# # Calculate accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy on test set: {accuracy:.2f}")


Accuracy on test set: 0.65


In [35]:
# from sklearn.linear_model import LogisticRegression
# df['Description'] = df['Description'].astype(str).fillna('')
# # df['Description'] = df['Description'].apply(preprocess_text)

# # Vectorize the text using CountVectorizer
# vectorizer = CountVectorizer(max_features=8)
# X = vectorizer.fit_transform(df['Description'])
# y = df['Category']

# # Train-test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# # Train the model
# model = LogisticRegression(random_state=2)
# model.fit(X_train, y_train)
# # Evaluate the model
# y_pred = model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy on test set: {accuracy:.2f}")

Accuracy on test set: 0.64


In [38]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# # Assuming df is your DataFrame
# df['Description'] = df['Description'].astype(str).fillna('')
# # df['Description'] = df['Description'].apply(preprocess_text)

# # Vectorize the text using CountVectorizer
# vectorizer = CountVectorizer(max_features=8)
# X = vectorizer.fit_transform(df['Description'])
# y = df['Category']

# # Train-test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# # Train the model
# model = DecisionTreeClassifier(random_state=2)
# model.fit(X_train, y_train)

# # Evaluate the model
# y_pred = model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy on test set: {accuracy:.2f}")


Accuracy on test set: 0.64


In [39]:
# from sklearn.svm import SVC
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# # Assuming df is your DataFrame
# df['Description'] = df['Description'].astype(str).fillna('')
# # df['Description'] = df['Description'].apply(preprocess_text)

# # Vectorize the text using CountVectorizer
# vectorizer = CountVectorizer(max_features=8)
# X = vectorizer.fit_transform(df['Description'])
# y = df['Category']

# # Train-test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# # Train the model
# model = SVC(probability=True, random_state=2)
# model.fit(X_train, y_train)

# # Evaluate the model
# y_pred = model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy on test set: {accuracy:.2f}")


Accuracy on test set: 0.64


In [40]:
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# # Assuming df is your DataFrame
# df['Description'] = df['Description'].astype(str).fillna('')
# # df['Description'] = df['Description'].apply(preprocess_text)

# # Vectorize the text using CountVectorizer
# vectorizer = CountVectorizer(max_features=8)
# X = vectorizer.fit_transform(df['Description'])
# y = df['Category']

# # Train-test split
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)

# # Train the model
# model = GradientBoostingClassifier(random_state=2)
# model.fit(X_train, y_train)

# # Evaluate the model
# y_pred = model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy on test set: {accuracy:.2f}")


Accuracy on test set: 0.64
